# Data Analysis Tools-Assignment 2([Coursera](https://www.coursera.org/learn/data-analysis-tools))
# Chi Square Test(Bonferroni Adjustment Post Hoc)

In [49]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.color_palette("colorblind")
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import gc

from itertools import combinations

import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi
from statsmodels.stats.multitest import multipletests

In [50]:
#-------Read Data-------
df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv',parse_dates=['datetime'])
df = df.rename({'count':'count_of_rentals'},axis=1)
df['month'] = df['datetime'].dt.month
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count_of_rentals,month
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,1


In [52]:
df['weather'] = df['weather'].map({ 
    1:'Clear, Few clouds, Partly cloudy, Partly cloudy',\
    2:'Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist',\
    3:'Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds',\
    4:'Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog'
}) 

**Let's consider the categorical explanatory variable 'Season' and the response variable The Count of rentals.**

In [55]:
#--------------Crosstab------------------
#---------Caegorical to Categorical------
pd.crosstab(df['month'], df['weather'])

weather,"Clear, Few clouds, Partly cloudy, Partly cloudy","Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog","Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds","Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist"
month,,,,
1,592,1,60,231
2,576,0,88,237
3,591,0,63,247
4,592,0,76,241
5,528,0,97,287
6,681,0,51,180
7,718,0,39,155
8,654,0,63,195
9,558,0,97,254


In [56]:
#--------Crosstab(Proportions)-----------
#---------Caegorical to Categorical------
pd.crosstab(df['month'], df['weather']).apply(lambda r: r/r.sum(), axis=0)

weather,"Clear, Few clouds, Partly cloudy, Partly cloudy","Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog","Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds","Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist"
month,,,,
1,0.082314,1.0,0.069849,0.081510
2,0.080089,0.0,0.102445,0.083627
3,0.082175,0.0,0.073341,0.087156
4,0.082314,0.0,0.088475,0.085039
5,0.073415,0.0,0.112922,0.101270
6,0.094689,0.0,0.059371,0.063514
7,0.099833,0.0,0.045402,0.054693
8,0.090934,0.0,0.073341,0.068807
9,0.077586,0.0,0.112922,0.089626


## Chi Square Test of Significance
A Chi square test helps determine if there is any significant relationship between two categorical variables.

In [61]:
print ('chi-square value, p value, expected counts',end=":\n")
cs1= scipy.stats.chi2_contingency(pd.crosstab(df['weather'], df['month']))
print (cs1)

chi-square value, p value, expected counts:
(247.5603815136428, 1.0505381541371918e-34, 33, array([[5.84027926e+02, 5.95259232e+02, 5.95259232e+02, 6.00544553e+02,
        6.02526548e+02, 6.02526548e+02, 6.02526548e+02, 6.02526548e+02,
        6.00544553e+02, 6.01865883e+02, 6.01865883e+02, 6.02526548e+02],
       [8.12052177e-02, 8.27668565e-02, 8.27668565e-02, 8.35017454e-02,
        8.37773287e-02, 8.37773287e-02, 8.37773287e-02, 8.37773287e-02,
        8.35017454e-02, 8.36854676e-02, 8.36854676e-02, 8.37773287e-02],
       [6.97552820e+01, 7.10967297e+01, 7.10967297e+01, 7.17279993e+01,
        7.19647253e+01, 7.19647253e+01, 7.19647253e+01, 7.19647253e+01,
        7.17279993e+01, 7.18858166e+01, 7.18858166e+01, 7.19647253e+01],
       [2.30135587e+02, 2.34561271e+02, 2.34561271e+02, 2.36643946e+02,
        2.37424949e+02, 2.37424949e+02, 2.37424949e+02, 2.37424949e+02,
        2.36643946e+02, 2.37164615e+02, 2.37164615e+02, 2.37424949e+02]]))


### Chi Square Test Inference
The p-value of the test is 1.0505381541371918e-34(approximately 0), so we fail to accept the Null Hypothesis and infer that there is some relationship between Month of the Year and Weather.

# Bonferroni Adjustment for Chi Square(Post-Hoc)
Since we have multiple pairs across the two Catgorical levels(Pairs of Month and Weather), hence we do have a chance of rejecting Null Hypothesis when it actually true, essentially commiting a **Type-1 Error**. What if there is one categorical pair for which the proportion difference is significant.  A way to mitigate this is to use Bonferroni **post Chi Square Adjustement**. We compute the Chi square for multiple categorical pairs and our inference is based on a :<br>**new significance level = $\alpha$/number of comparisons**(where $\alpha$ is most of the times **0.05**). 

Inspired by [https://neuhofmo.github.io/chi-square-and-post-hoc-in-python/](http://)

In [62]:
#-------------------------Post Hoc Test Chi Square--------------------

# Store p-values of each pair of month
p_vals_chi = []
pairs_of_months = list(combinations(df['month'].unique(),2))

#For Each Pair of Months compute Chi Square Stats
for each_pair in pairs_of_months:
    each_df = df[(df['month']==each_pair[0]) | (df['month']==each_pair[1])]
    p_vals_chi.append(\
          scipy.stats.chi2_contingency(
            pd.crosstab(each_df['weather'], each_df['month']))[1]
         )
    
gc.collect()

4861

In [63]:
#Results of Bonferroni Adjustment
bonferroni_results = pd.DataFrame(columns=['pair of months',\
                                           'original p value',\
                                           'corrected p value',\
                                           'Reject Null?'])

bonferroni_results['pair of months'] = pairs_of_months
bonferroni_results['original p value'] = p_vals_chi

#Perform Bonferroni on the p-values and get the reject/fail to reject Null Hypothesis result.
multi_test_results_bonferroni = multipletests(p_vals_chi, method='bonferroni')

bonferroni_results['corrected p value'] = multi_test_results_bonferroni[1]
bonferroni_results['Reject Null?'] = multi_test_results_bonferroni[0]

bonferroni_results.head()

,pair of months,original p value,corrected p value,Reject Null?
0,"(1, 2)",0.092380,1.000000,False
1,"(1, 3)",0.694368,1.000000,False
2,"(1, 4)",0.432438,1.000000,False
3,"(1, 5)",0.000274,0.018053,True
4,"(1, 6)",0.003120,0.205952,False


In [64]:
print(f"{bonferroni_results[bonferroni_results['Reject Null?']==True].shape[0]} pairs of months\
 have a signifant relationship w.r.t Weather",end=".\n")

print(f"{bonferroni_results[bonferroni_results['Reject Null?']==False].shape[0]} pairs of months\
 do not have a signifant relationship w.r.t Weather",end=".")

32 pairs of months have a signifant relationship w.r.t Weather.
34 pairs of months do not have a signifant relationship w.r.t Weather.

### Bonferroni Adjusted Chi Square Test Inference

For nearly half of the pairs, we have the null hypotheis rejected and for a similar number of pairs we fail to reject the null hpothesis. Let us have some examples to illustrate this.<br> The month pair(1,2) i.e **January and February** dont vary much in terms of weather and the Null Hypothesis is not rejected.<br> However for months (1,5) - **January and May** the Null Hypothesis has been rejected, we can infer that they do vary in terms of weather(possibly **Light Snow/Rain** is significantly more during the month of May as compared to January). 